# Parallelizing image convolution

## Learning Goals

By the end of this tutorial, you will be able to:

- Employ three parallelization libraries to speed up a serial process.
- Calculate the speedup of the different approaches shown.
- Evaluate which library is suited to your task.

## Introduction

This notebook shows how to speed up an image convolution task using these three libraries:

* Ray: an open-source unified compute framework that makes it easy to scale AI and Python workloads.
* Multiprocessing: part of the standard library; supports spawning processes using an API similar to the threading module; offers both local and remote concurrency, effectively side-stepping the Global Interpreter Lock by using subprocesses instead of threads.
* Dask: developed to natively scale computational packages like numpy, pandas and scikit-learn, and the surrounding ecosystem, to multi-core machines and distributed clusters when datasets exceed memory.

## Imports

* _multiprocessing.Pool_ for multiprocessing using the standard library
* _time_ for timing the processes
* _dask.distributed.Client_ for making a local Dask cluster
* _numpy_ and _scipy.signal_ for numerical work
* _psutil_ for finding the available processors on your machine
* _ray_ for scaling up Python tasks

In [1]:
from multiprocessing import Pool
import time

from dask.distributed import Client
import numpy as np
import psutil
import scipy.signal
import ray

2024-08-30 19:49:13,290	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


## Find the cpus available

Find and print the number of cpus
(taken from https://towardsdatascience.com/10x-faster-parallel-python-without-python-multiprocessing-e5017c93cce1)

In [2]:
num_cpus = psutil.cpu_count(logical=True)
print(num_cpus)

4


## Process serially using a conventional loop

Use `scipy.signal` to convolve two 2-dimensional arrays and return a 5x5 downsampled result.

In [3]:
def fconv(image, random_filter):
    return scipy.signal.convolve2d(image, random_filter)[::5, ::5]

In [4]:
filters = [np.random.normal(size=(4, 4)) for _ in range(num_cpus)]

Process 100 iterations serially, then extrapolate to num_cpus*100

In [5]:
start = time.time()
num_iter = 100
image = np.zeros((3000, 3000))
for i in range(num_iter):
    result = fconv(image, filters[i % num_cpus])
duration_conv = time.time() - start
print("(scaled) conventional duration for {:d} iterations = {:.1f} seconds"
      .format(num_cpus*num_iter, duration_conv*num_cpus))

(scaled) conventional duration for 400 iterations = 148.5 seconds


## Process in parallel using Ray

[Documentation for ray](https://docs.ray.io/en/latest/)

The warning raised by `ray.init` only affects shared object usage, which is not an issue for this tutorial. It may harm performance in other scenarios.

In [6]:
ray.init(num_cpus=num_cpus)

2024-08-30 19:49:52,381	INFO worker.py:1783 -- Started a local Ray instance.


Python version:,3.11.9
Ray version:,2.35.0


Use `scipy.signal` to convolve two 2-dimensional arrays and return a 5x5 downsampled result. To use Ray, we decorate the function that is doing the work.

In [7]:
@ray.remote
def fray(image, random_filter):
    return scipy.signal.convolve2d(image, random_filter)[::5, ::5]

In the following loop, `ray.put` places the image into shared memory. The call to `ray.get` retrieves the result.

In [8]:
start = time.time()
image = np.zeros((3000, 3000))
for _ in range(100):
    image_id = ray.put(image)
    ray.get([fray.remote(image_id, filters[i]) for i in range(num_cpus)])
duration_ray = time.time() - start
print("Ray duration = {:.1f}, speedup = {:.2f}"
      .format(duration_ray, duration_conv*num_cpus / duration_ray))

Ray duration = 86.2, speedup = 1.72


In [9]:
ray.shutdown()

## Process in parallel using multiprocessing

[Documentation for multiprocessing](https://docs.python.org/3/library/multiprocessing.html)

Use `scipy.signal` to convolve two 2-dimensional arrays and return a 5x5 downsampled result. The call to the function has a slightly different form than that for the serial loop.

In [10]:
def fmp(args):
    image, random_filter = args
    return scipy.signal.convolve2d(image, random_filter)[::5, ::5]

Use a multiprocessing pool with the number of cpus we found earlier.

In [11]:
pool = Pool(num_cpus)

Using `pool.map` is the closest analog in multiprocessing to the Ray API.

In [12]:
start = time.time()
image = np.zeros((3000, 3000))
for _ in range(100):
    pool.map(fmp, zip(num_cpus * [image], filters))
duration_mp = time.time() - start
print("Multiprocessing duration = {:.1f}, speedup = {:.2f}"
      .format(duration_mp, duration_conv*num_cpus / duration_mp))

Multiprocessing duration = 141.1, speedup = 1.05


## Process using Dask

[Documentation for Dask](https://www.dask.org/get-started)

Define a Dask distributed client with number of workers set to the number of cpus we found earlier, and with one thread per worker.

In [13]:
client = Client(n_workers=num_cpus, threads_per_worker=1)

In [14]:
print(client)

<Client: 'tcp://127.0.0.1:35347' processes=4 threads=4, memory=15.61 GiB>


Dask recommends scattering the large inputs across the workers, though this makes little difference in execution time.

In [15]:
start = time.time()
image = np.zeros((3000, 3000))
for _ in range(100):
    for j in range(num_cpus):
        big_future = client.scatter((image, filters[j % num_cpus]))
        future = client.submit(fconv, big_future)
duration_dask = time.time() - start
print("Dask duration = {:.1f}, speedup = {:.2f}"
      .format(duration_dask, duration_conv*num_cpus / duration_dask))

2024-08-30 19:53:44,187 - distributed.worker - ERROR - Compute Failed
Key:       fconv-2525d9ae395e966ab6a38c698eb12e02
State:     executing
Function:  fconv
args:      ((array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]]), array([[-0.79762753, -1.34360677, -0.65071631, -0.90866086],
       [-1.12844665,  1.66289612, -1.43863742,  0.4767091 ],
       [-1.85043238,  1.20934311,  0.04252848, -0.20778521],
       [ 0.93728218,  1.56554166,  0.79808976,  1.6689882 ]])))
kwargs:    {}
Exception: 'TypeError("fconv() missing 1 required positional argument: \'random_filter\'")'
Traceback: ''

2024-08-30 19:53:44,303 - distributed.worker - ERROR - Compute Failed
Key:       fconv-52e397cd09706d537861ef5ae5ed6207
State:     executing
Function:  fconv
args:      ((array([[0., 0., 0., ..., 0., 0., 0.],
       [0.

2024-08-30 19:53:44,395 - distributed.worker - ERROR - Compute Failed
Key:       fconv-0eb6f34ab727191253b4cdf9e773bd7f
State:     executing
Function:  fconv
args:      ((array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]]), array([[-1.64543502,  0.90111577,  0.29983251, -0.65971244],
       [ 1.52689482,  0.23925744,  0.31863011,  0.26720248],
       [-1.28896017, -0.18718218, -0.36747218, -1.05855011],
       [-0.37959085, -0.17167469, -0.9705473 , -1.47948348]])))
kwargs:    {}
Exception: 'TypeError("fconv() missing 1 required positional argument: \'random_filter\'")'
Traceback: ''

2024-08-30 19:53:44,489 - distributed.worker - ERROR - Compute Failed
Key:       fconv-8fdadcd84f92b54026c0590f2eea9165
State:     executing
Function:  fconv
args:      ((array([[0., 0., 0., ..., 0., 0., 0.],
       [0.

2024-08-30 19:53:44,674 - distributed.worker - ERROR - Compute Failed
Key:       fconv-f73a39a44edaec727ba9d739f9984c28
State:     executing
Function:  fconv
args:      ((array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]]), array([[-1.73125328,  1.67828851,  0.83733651, -1.17971026],
       [-0.72086117, -0.00947245, -0.14862206,  1.36430793],
       [ 0.41843088, -2.09702697,  0.20977004, -1.93666342],
       [-0.09211273, -0.77981353,  0.36446084, -0.54897659]])))
kwargs:    {}
Exception: 'TypeError("fconv() missing 1 required positional argument: \'random_filter\'")'
Traceback: ''

2024-08-30 19:53:44,771 - distributed.worker - ERROR - Compute Failed
Key:       fconv-ae85dcd4df7e17f4f2b0697c730eff0c
State:     executing
Function:  fconv
args:      ((array([[0., 0., 0., ..., 0., 0., 0.],
       [0.

2024-08-30 19:53:44,949 - distributed.worker - ERROR - Compute Failed
Key:       fconv-a1d45fc64ee3c2cbc80b095f9e482ce1
State:     executing
Function:  fconv
args:      ((array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]]), array([[-0.79762753, -1.34360677, -0.65071631, -0.90866086],
       [-1.12844665,  1.66289612, -1.43863742,  0.4767091 ],
       [-1.85043238,  1.20934311,  0.04252848, -0.20778521],
       [ 0.93728218,  1.56554166,  0.79808976,  1.6689882 ]])))
kwargs:    {}
Exception: 'TypeError("fconv() missing 1 required positional argument: \'random_filter\'")'
Traceback: ''

2024-08-30 19:53:45,036 - distributed.worker - ERROR - Compute Failed
Key:       fconv-f0f96a038c555e5149129ec76b6d3269
State:     executing
Function:  fconv
args:      ((array([[0., 0., 0., ..., 0., 0., 0.],
       [0.

2024-08-30 19:53:45,219 - distributed.worker - ERROR - Compute Failed
Key:       fconv-ec66693b52265b0c992e0af7e291d7a5
State:     executing
Function:  fconv
args:      ((array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]]), array([[ 0.32643043,  0.85264523,  0.91809302, -1.26267228],
       [-0.37360246,  1.02145752, -0.9946663 ,  0.86984413],
       [ 1.06252105, -0.26859107,  0.36020379,  0.23642664],
       [ 0.10690739,  0.95110907, -1.20269699, -0.55835212]])))
kwargs:    {}
Exception: 'TypeError("fconv() missing 1 required positional argument: \'random_filter\'")'
Traceback: ''

2024-08-30 19:53:45,309 - distributed.worker - ERROR - Compute Failed
Key:       fconv-7d39641658f9ff197848f0f708711f14
State:     executing
Function:  fconv
args:      ((array([[0., 0., 0., ..., 0., 0., 0.],
       [0.

...

2024-08-30 19:54:19,487 - distributed.worker - ERROR - Compute Failed
Key:       fconv-07e1db483b3b39e4d21b592edd4949f4
State:     executing
Function:  fconv
args:      ((array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]]), array([[-1.73125328,  1.67828851,  0.83733651, -1.17971026],
       [-0.72086117, -0.00947245, -0.14862206,  1.36430793],
       [ 0.41843088, -2.09702697,  0.20977004, -1.93666342],
       [-0.09211273, -0.77981353,  0.36446084, -0.54897659]])))
kwargs:    {}
Exception: 'TypeError("fconv() missing 1 required positional argument: \'random_filter\'")'
Traceback: ''

2024-08-30 19:54:19,588 - distributed.worker - ERROR - Compute Failed
Key:       fconv-dcb32db68207fe1f9c33d9d32e9b1a98
State:     executing
Function:  fconv
args:      ((array([[0., 0., 0., ..., 0., 0., 0.],
       [0.

2024-08-30 19:54:19,771 - distributed.worker - ERROR - Compute Failed
Key:       fconv-65477e5198021cd61005d58c2e735707
State:     executing
Function:  fconv
args:      ((array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]]), array([[-0.79762753, -1.34360677, -0.65071631, -0.90866086],
       [-1.12844665,  1.66289612, -1.43863742,  0.4767091 ],
       [-1.85043238,  1.20934311,  0.04252848, -0.20778521],
       [ 0.93728218,  1.56554166,  0.79808976,  1.6689882 ]])))
kwargs:    {}
Exception: 'TypeError("fconv() missing 1 required positional argument: \'random_filter\'")'
Traceback: ''

2024-08-30 19:54:19,855 - distributed.worker - ERROR - Compute Failed
Key:       fconv-a2f7d7f1d3a0f237ae3aec1722252972
State:     executing
Function:  fconv
args:      ((array([[0., 0., 0., ..., 0., 0., 0.],
       [0.

2024-08-30 19:54:20,037 - distributed.worker - ERROR - Compute Failed
Key:       fconv-6c9bbe353646542be4d6c12b83815dbf
State:     executing
Function:  fconv
args:      ((array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]]), array([[ 0.32643043,  0.85264523,  0.91809302, -1.26267228],
       [-0.37360246,  1.02145752, -0.9946663 ,  0.86984413],
       [ 1.06252105, -0.26859107,  0.36020379,  0.23642664],
       [ 0.10690739,  0.95110907, -1.20269699, -0.55835212]])))
kwargs:    {}
Exception: 'TypeError("fconv() missing 1 required positional argument: \'random_filter\'")'
Traceback: ''

2024-08-30 19:54:20,141 - distributed.worker - ERROR - Compute Failed
Key:       fconv-514de7e2223934bbbb9cb08d2c5e7f82
State:     executing
Function:  fconv
args:      ((array([[0., 0., 0., ..., 0., 0., 0.],
       [0.

2024-08-30 19:54:20,313 - distributed.worker - ERROR - Compute Failed
Key:       fconv-adee73690c69ed2f61e51921d791ef99
State:     executing
Function:  fconv
args:      ((array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]]), array([[-1.64543502,  0.90111577,  0.29983251, -0.65971244],
       [ 1.52689482,  0.23925744,  0.31863011,  0.26720248],
       [-1.28896017, -0.18718218, -0.36747218, -1.05855011],
       [-0.37959085, -0.17167469, -0.9705473 , -1.47948348]])))
kwargs:    {}
Exception: 'TypeError("fconv() missing 1 required positional argument: \'random_filter\'")'
Traceback: ''

2024-08-30 19:54:20,399 - distributed.worker - ERROR - Compute Failed
Key:       fconv-881b4881ea8a3b3b0ab4d0117f7a6948
State:     executing
Function:  fconv
args:      ((array([[0., 0., 0., ..., 0., 0., 0.],
       [0.

Dask duration = 36.9, speedup = 4.02


In [16]:
client.close()

2024-08-30 19:54:20,545 - distributed.worker - ERROR - Failed to communicate with scheduler during heartbeat.
Traceback (most recent call last):
  File "/home/runner/work/irsa-tutorials/irsa-tutorials/.tox/py311-buildhtml/lib/python3.11/site-packages/distributed/comm/tcp.py", line 225, in read
    frames_nosplit_nbytes_bin = await stream.read_bytes(fmt_size)
                                ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
tornado.iostream.StreamClosedError: Stream is closed

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/home/runner/work/irsa-tutorials/irsa-tutorials/.tox/py311-buildhtml/lib/python3.11/site-packages/distributed/worker.py", line 1250, in heartbeat
    response = await retry_operation(
               ^^^^^^^^^^^^^^^^^^^^^^
  File "/home/runner/work/irsa-tutorials/irsa-tutorials/.tox/py311-buildhtml/lib/python3.11/site-packages/distributed/utils_comm.py", line 459, in retry_operation
    return await retr

## Conclusions

* Ray is the most effective at speeding up the convolution workload by fully utilizing all available processes
* Multiprocessing is second in effectiveness
* Dask delivers the least speedup; perhaps due to having only six processes on the dask.distributed client

## About this notebook

This notebook was developed by David Shupe (shupe@ipac.caltech.edu) in conjunction with Jessica Krick and the IRSA Science Platform team at IPAC.

## Citations

If you use these software packages in your work, please use the following citations:

* Dask: Dask Development Team (2016). Dask: Library for dynamic task scheduling. URL https://dask.org
* Ray: The Ray Development Team. URL https://docs.ray.io